In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import rasterio as rio
import rioxarray as rxr


In [2]:
city = 'MILANO'
#prediction_name = 'UHI_ANN_sim_veg_20_random_parks_15000_20000'#'UHI_ANN_sim_veg2'
prediction_name = 'UHII_green_corridor_5m'
base_folder = f'predictions'
data_folder = "data"

file_name = f'{city}_{prediction_name}.csv'
prob = pd.read_csv(f'{base_folder}/{file_name}', index_col=0)

# get raster parameters
#landcover_path = f"{data_folder}/landcover/DUSAF_2021_milano_mapped.tif"
#landcover_path = f"{data_folder}/MILANO_landcover.tif"
landcover_path = f"{data_folder}/landcover/DUSAF_2021_MCM_mapped.tif"
#landcover_path = '/home/user/ODC_harmonia/Landsat/Milan/data/clip/LC08_L2SP_194028_20220810_20220818_02_T1/LC08_L2SP_194028_20220810_20220818_02_T1_LST.TIF'
with rio.open(landcover_path, driver="GTiff") as base_raster:
    transform = base_raster.transform
    init_x = transform[2]
    init_y = transform[5]
    step_x = transform[0]
    step_y = transform[4]
    raster_w = base_raster.width
    raster_h = base_raster.height
    profile = base_raster.profile
    
transform

Affine(5.0, 0.0, 477051.11479999963,
       0.0, -5.0, 5054450.992000001)

In [3]:
prob

,probability,x,y
0,5.2282,513016.1148,5042500.992
1,5.2253,513021.1148,5042500.992
2,5.2230,513026.1148,5042500.992
3,5.2211,513031.1148,5042500.992
4,5.2193,513036.1148,5042500.992
...,...,...,...
7201800,-1.7842,515101.1148,5025940.992
7201801,-1.7781,515106.1148,5025940.992
7201802,-2.3797,515081.1148,5025935.992
7201803,-2.3781,515086.1148,5025935.992


In [4]:
da = prob.set_index(['y', 'x']).to_xarray()
da = da.astype('float64')
da["x"] = da["x"].astype("int32")
da["y"] = da["y"].astype("int32")
da

<xarray.Dataset>
Dimensions:      (y: 3314, x: 3708)
Coordinates:
  * y            (y) int32 5025935 5025940 5025945 ... 5042490 5042495 5042500
  * x            (x) int32 503176 503181 503186 503191 ... 521701 521706 521711
Data variables:
    probability  (y, x) float64 nan nan nan nan nan nan ... nan nan nan nan nan

In [5]:
# Generate the full grid with the base raster parameters
full_grid_ds = xr.Dataset(
    coords=dict(
        y=np.arange(init_y, init_y + (step_y * raster_h), step_y, dtype="int32"),
        x=np.arange(init_x, init_x + (step_x * raster_w), step_x, dtype="int32")
    )
)
full_grid_ds

<xarray.Dataset>
Dimensions:  (y: 10691, x: 13202)
Coordinates:
  * y        (y) int32 5054450 5054445 5054440 ... 5001010 5001005 5001000
  * x        (x) int32 477051 477056 477061 477066 ... 543046 543051 543056
Data variables:
    *empty*

In [6]:
print(full_grid_ds["x"].values[0:100])
print()
print(da["x"].values[0:100])

print(full_grid_ds["x"].values[4] == da["x"].values[0])
print(full_grid_ds["x"].values[4], da["x"].values[0])

[477051 477056 477061 477066 477071 477076 477081 477086 477091 477096
 477101 477106 477111 477116 477121 477126 477131 477136 477141 477146
 477151 477156 477161 477166 477171 477176 477181 477186 477191 477196
 477201 477206 477211 477216 477221 477226 477231 477236 477241 477246
 477251 477256 477261 477266 477271 477276 477281 477286 477291 477296
 477301 477306 477311 477316 477321 477326 477331 477336 477341 477346
 477351 477356 477361 477366 477371 477376 477381 477386 477391 477396
 477401 477406 477411 477416 477421 477426 477431 477436 477441 477446
 477451 477456 477461 477466 477471 477476 477481 477486 477491 477496
 477501 477506 477511 477516 477521 477526 477531 477536 477541 477546]

[503176 503181 503186 503191 503196 503201 503206 503211 503216 503221
 503226 503231 503236 503241 503246 503251 503256 503261 503266 503271
 503276 503281 503286 503291 503296 503301 503306 503311 503316 503321
 503326 503331 503336 503341 503346 503351 503356 503361 503366 503371
 503

In [7]:
#Combine the probability points with the grid for filling the complete raster matrix
arr1 = da
arr2 = full_grid_ds

combined_arr = arr2.combine_first(arr1)
combined_arr = combined_arr.fillna(9999)
combined_arr = combined_arr.astype('float64')
combined_arr

<xarray.Dataset>
Dimensions:      (y: 10691, x: 13202)
Coordinates:
  * y            (y) int32 5001000 5001005 5001010 ... 5054440 5054445 5054450
  * x            (x) int32 477051 477056 477061 477066 ... 543046 543051 543056
Data variables:
    probability  (y, x) float64 9.999e+03 9.999e+03 ... 9.999e+03 9.999e+03

In [8]:
#copy raster profile from the landcover
prediction_profile = profile.copy()
prediction_profile['dtype'] = 'float32'
prediction_profile

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': 9999.0, 'width': 13202, 'height': 10691, 'count': 1, 'crs': CRS.from_epsg(32632), 'transform': Affine(5.0, 0.0, 477051.11479999963,
       0.0, -5.0, 5054450.992000001), 'tiled': False, 'interleave': 'band'}

In [9]:
raster_path = f'rasters/{prediction_name}.tif'
# the array is initially flipped in x and y axis.
array_to_write = np.flip(np.flip(combined_arr.probability.to_numpy()), axis=1)
#array_to_write = np.flip(combined_arr.probability.to_numpy(), axis=0)

with rio.open(raster_path, 'w', **prediction_profile) as dest:
    dest.write(array_to_write, 1)

In [10]:
print(array_to_write.shape, (prediction_profile['height'], prediction_profile['width']))


(10691, 13202) (10691, 13202)
